In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [2]:
state = 'ohio'

In [3]:
pop = pd.read_csv('population.csv')
state_path = '05_MME_Final/'+ state + '.parquet'
state_MME = pd.read_parquet(state_path)

In [4]:
per_capita_MME = state_MME.copy()
per_capita_MME['county_population'] = pd.NA
per_capita_MME['per_capita_MME_g'] = pd.NA
for index, row in state_MME.iterrows():
    FIPS = row['FIPS']
    year = row['TransactionYear']
    target = pop[pop['FIPS'] == FIPS]
    if len(target==1):
        population = int(target[f'{year}'].values)
        #print(population)
        idx = per_capita_MME[(per_capita_MME['TransactionYear'] == year)&
                                        (per_capita_MME['FIPS'] == FIPS)].index
        per_capita_MME.loc[idx,'county_population'] = population
    else:
        print(f'Did not find data for FIPS: {FIPS}, year: {year}')

per_capita_MME['per_capita_MME_g'] = per_capita_MME['morphine_equivalent_g'] / per_capita_MME['county_population']
print(per_capita_MME.sample(5))
assert per_capita_MME[per_capita_MME['per_capita_MME_g'].isna()].empty, 'There is missing per_capita_MME_g'
assert per_capita_MME[per_capita_MME['county_population'].isna()].empty, 'There is missing county_population'

    BUYER_STATE BUYER_COUNTY  TransactionYear  MME_Conversion_Factor  \
511          OH       MEDINA             2007                13730.5   
867          OH         WOOD             2013                11856.5   
328          OH       HARDIN             2014                 3079.5   
568          OH   MONTGOMERY             2014                66686.0   
109          OH    CHAMPAIGN             2015                 3700.5   

     CALC_BASE_WT_IN_GM  morphine_equivalent_g   FIPS county_population  \
511        34170.130602           45252.844291  39103            168775   
867        21840.790405           29172.661972  39173            129073   
328         6947.249978            9166.864541  39065             31780   
568       157667.915169          201240.970289  39113            532708   
109         8671.582278           11810.134377  39021             38976   

    per_capita_MME_g  
511         0.268125  
867         0.226017  
328         0.288448  
568          0.37777  
1

In [5]:
# Step 4: Save the per capita dataframe back to a Parquet file
final_path = '06_PerCapitaMME/' + state + '_PC.parquet'
per_capita_MME.to_parquet(final_path)